In [3]:
import pandas as pd

In [4]:
%config Completer.use_jedi = False

In [5]:
from ast import literal_eval

In [6]:
from collections import Counter

In [7]:
from utils.basic_tokenize import tokenize

In [21]:
from gensim.utils import simple_preprocess

In [8]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [9]:
df = pd.read_csv(r"D:\data\openparliament\df_climate.csv")
df.drop('Unnamed: 0', axis=1, inplace=True)

In [10]:
df['text_en']= df.text_en.apply(lambda x: x.replace('Mr. Speaker,',' '))

In [ ]:
df['tokens'] = df.tokenized_content.apply(literal_eval)

In [22]:
%%time
df['tokens'] = df.text_en.apply(lambda x: simple_preprocess(x, deacc=True))

Wall time: 5.46 s


In [27]:
def bigrams(sentence):
    part1 = [i+ "_"+j for i,j in zip(sentence[::2], sentence[1::2])]
    part2 = [i+ "_"+j for i,j in zip(sentence[1::2], sentence[2::2])]
    
    part1.extend(part2)
    return part1

In [28]:
a = ['a','b','c']
b = [1,2,3]

In [29]:
list(zip(a,b))

[('a', 1), ('b', 2), ('c', 3)]

In [30]:
df['bigrams'] = df.tokens.apply(bigrams)

In [31]:
df.bigrams.head()

0    [last_week, more_than, six_million, canadians_...
1    [last_week, more_than, six_million, canadians_...
2    [last_week, more_than, six_million, canadians_...
3    [it_is, privilege_to, be_able, to_speak, to_th...
4    [it_is, privilege_to, be_able, to_speak, to_th...
Name: bigrams, dtype: object

In [32]:
bigram_counts = dict()
template_counter = Counter()
pids = list(set(df.politician_id))
for pol in pids:
    bigram_counts[pol] = template_counter.copy()

In [33]:
def update_counts(row):
    bigrams = row.loc['bigrams']
    if bigram_counts.get(row.loc['politician_id'])==None:
        bigram_counts[row.loc['politician_id']] = template_counter.copy()
        bigram_counts[row.loc['politician_id']].update(bigrams)
    else:
        bigram_counts[row.loc['politician_id']].update(bigrams)

In [34]:
df.apply(update_counts,axis=1)

0        None
1        None
2        None
3        None
4        None
         ... 
19655    None
19656    None
19657    None
19658    None
19659    None
Length: 19660, dtype: object

note: 'mr_speaker' needs to be removed from the bigrams, it's such a stop phrase

In [70]:
for key, counter in bigram_counts.items():
    print('politician #:')
    print(counter.most_common(4))

politician #:
[('we_are', 3), ('of_the', 2), ('this_process', 2), ('mr_speaker', 1)]
politician #:
[('of_the', 8), ('and_air', 8), ('new_government', 8), ('canada_new', 8)]
politician #:
[('will_be', 8), ('mr_speaker', 4), ('the_member', 4), ('happy_to', 4)]
politician #:
[('carbon_tax', 22), ('the_liberal', 18), ('mr_speaker', 16), ('prime_minister', 12)]
politician #:
[('mr_speaker', 452), ('of_the', 184), ('we_are', 176), ('to_the', 148)]
politician #:
[('of_the', 52), ('carbon_tax', 52), ('federal_government', 36), ('the_federal', 36)]
politician #:
[('mr_speaker', 158), ('we_are', 106), ('we_have', 94), ('of_the', 86)]
politician #:
[('blue_green', 21), ('green_algae', 21), ('this_bill', 15), ('the_government', 12)]
politician #:
[('of_the', 29), ('it_is', 25), ('we_are', 24), ('the_government', 19)]
politician #:
[('in_the', 12), ('of_the', 12), ('mr_speaker', 8), ('st_lawrence', 8)]
politician #:
[('energy_efficiency', 3), ('our_climate', 2), ('meet_our', 2), ('climate_change', 

In [75]:
df[~df.text_en.str.contains("Mr. Speaker")].shape

(1572, 32)

In [76]:
df[df.text_en.str.contains("Mr. Speaker")].shape

(18088, 32)

In [77]:
df.shape

(19660, 32)